In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 💼 The Labor-Leisure Choice Model

How do individuals decide how much time to spend working versus enjoying leisure? This fundamental microeconomic model analyzes the trade-off between consuming goods (which requires earning income through labor) and enjoying leisure time.

Individuals aim to maximize their utility (satisfaction) derived from consumption and leisure, subject to the constraints imposed by time and the market wage rate.

# ⚙️ Model Setup

1.  **Time Endowment (T):** Individuals have a fixed amount of total time available (e.g., 24 hours per day), which they allocate between labor ($L$) and leisure ($\ell$).
    $$T = L + \ell$$

2.  **Budget Constraint:** Consumption ($c$) is financed solely by labor income. The amount an individual can consume is equal to their wage rate ($w$) multiplied by the hours they work ($L$). Substituting $L = T - \ell$:
    $$c = w L = w (T - \ell)$$
    This equation represents the **budget line**. Its slope is $-w$, indicating that to gain one more hour of leisure, the individual must give up $w$ units of consumption (the opportunity cost of leisure is the wage).

3.  **Preferences (Utility Function):** Individuals derive utility from both consumption ($c$) and leisure ($\ell$). We often use a utility function to represent these preferences. A common choice is the Cobb-Douglas form (often expressed in logs for convenience):
    $$U(c, \ell) = \ln(c) + \theta \ln(\ell)$$
    - $\theta$ (theta): A parameter representing the relative preference for leisure compared to consumption. A higher $\theta$ means the individual values leisure more strongly.

**The Goal:** The individual chooses the combination of consumption ($c$) and leisure ($\ell$) that maximizes their utility $U(c, \ell)$ while satisfying the budget constraint $c = w(T - \ell)$.

# ✨ Optimal Choice: Tangency Condition

The optimal choice occurs where the individual's **indifference curve** (representing combinations of $c$ and $\ell$ that yield the same level of utility) is just tangent to the **budget line**.

At the point of tangency, the slope of the indifference curve equals the slope of the budget line.

* **Slope of Indifference Curve:** This is the Marginal Rate of Substitution (MRS), which measures how much consumption the individual is *willing* to give up for one more unit of leisure while keeping utility constant.
    $$MRS = \frac{\text{Marginal Utility of Leisure}}{\text{Marginal Utility of Consumption}} = \frac{\partial U / \partial \ell}{\partial U / \partial c}$$
    For $U = \ln(c) + \theta \ln(\ell)$, the marginal utilities are $\frac{\partial U}{\partial \ell} = \frac{\theta}{\ell}$ and $\frac{\partial U}{\partial c} = \frac{1}{c}$. So:
    $$MRS = \frac{\theta / \ell}{1 / c} = \frac{\theta c}{\ell}$$

* **Slope of Budget Line:** This is the negative of the wage rate ($-w$), representing the market rate at which consumption *can* be traded for leisure.

**Optimality Condition (Tangency):**
$$MRS = w$$
$$\frac{\theta c}{\ell} = w$$

The simulation below finds the optimal bundle $(c^*, \ell^*)$ that satisfies both this tangency condition and the budget constraint, and visualizes the budget line and indifference curves.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider # Use IntSlider for T
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def labor_leisure_optimizer(w=15.0, T=24, theta=1.0):
    """
    Calculates and plots the optimal labor-leisure choice.

    Args:
        w (float): Real wage rate.
        T (int): Total time endowment.
        theta (float): Preference parameter for leisure in U=ln(c) + theta*ln(l).
    """
    # Input validation
    T = int(T)
    if T <= 0: T = 1
    w = max(w, 0.1)
    theta = max(theta, 0.01)

    # --- Analytical Solution ---
    # From MRS = w => theta * c / l = w
    # Substitute budget constraint c = w * (T - l):
    # theta * w * (T - l) / l = w
    # theta * (T - l) = l
    # theta * T - theta * l = l
    # theta * T = l * (1 + theta)
    # Optimal leisure l_star:
    l_star = (theta / (1 + theta)) * T
    # Ensure leisure doesn't exceed total time or become non-positive
    l_star = np.clip(l_star, 1e-6, T - 1e-6)

    # Optimal labor L_star:
    L_star = T - l_star
    # Optimal consumption c_star:
    c_star = w * L_star

    # --- Calculations for Plotting ---
    # Leisure values for plotting curves
    l_vals = np.linspace(1e-3, T - 1e-3, 400) # Avoid l=0 or l=T

    # Budget line: c = w * (T - l)
    c_budget = w * (T - l_vals)

    # Utility function
    def utility(c, l):
        # Use np.log where possible, handle non-positive values
        c_safe = np.maximum(c, 1e-9)
        l_safe = np.maximum(l, 1e-9)
        # Check for non-positive inputs that would lead to -inf
        if np.any(c <= 1e-9) or np.any(l <= 1e-9):
            # Handle array case: return array with -inf where needed
            u_val = np.full_like(c, -np.inf, dtype=float)
            valid_mask = (c > 1e-9) & (l > 1e-9)
            u_val[valid_mask] = np.log(c_safe[valid_mask]) + theta * np.log(l_safe[valid_mask])
            return u_val
        else:
            # Scalar case
            return np.log(c_safe) + theta * np.log(l_safe)


    # Utility level at the optimum
    U_star = utility(c_star, l_star)

    # Indifference curve at the optimum: U_star = ln(c) + theta*ln(l)
    # => ln(c) = U_star - theta*ln(l) => c = exp(U_star - theta*ln(l))
    with np.errstate(over='ignore'): # Ignore potential overflow for exp
        c_indiff_star = np.exp(U_star - theta * np.log(l_vals))

    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(10, 7))

    # Plot Budget Line
    ax.plot(l_vals, c_budget, label=f'Budget Line (Slope = -{w:.2f})', color='black', linewidth=2)

    # Plot Optimal Indifference Curve
    ax.plot(l_vals, c_indiff_star, linestyle='--', color='red', alpha=0.8, label=f'Optimal Indifference Curve (U≈{U_star:.2f})')

    # Plot Optimal Point (tangency)
    ax.scatter(l_star, c_star, color='red', s=120, zorder=5, label='Optimal Choice (c*, l*)')
    ax.annotate(f"Optimal Point\n Leisure l* = {l_star:.1f} hrs\n Labor L* = {L_star:.1f} hrs\n Cons. c* = {c_star:.1f}",
                xy=(l_star, c_star), xytext=(l_star + 0.05*T, c_star + 0.05*w*T),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.2'),
                fontsize=10, bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.7))

    # Add text for tangency condition
    # MRS = (theta * c) / l
    MRS_at_optimum = (theta * c_star) / l_star
    ax.text(0.05 * T, 0.9 * w * T, f"Optimality Check:\nMRS = θc*/l* = {MRS_at_optimum:.2f}\nWage w = {w:.2f}\n(MRS ≈ w: {'✅' if np.isclose(MRS_at_optimum, w) else '❌'})",
            fontsize=10, bbox=dict(boxstyle='round,pad=0.3', fc='lightyellow', ec='gray', alpha=0.8))


    ax.set_xlabel("Leisure (hours, l)")
    ax.set_ylabel("Consumption (units, c)")
    ax.set_title("Labor-Leisure Optimal Choice")
    ax.set_xlim(0, T)
    ax.set_ylim(0, w * T * 1.1) # Max possible consumption + 10%
    ax.legend(loc='upper right')
    ax.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # --- Display Results ---
    results_md = f"""
    ### 📊 Optimal Choice Results:

    * **Optimal Leisure (l\*):** {l_star:.2f} hours
    * **Optimal Labor (L\* = T - l\*):** {L_star:.2f} hours
    * **Optimal Consumption (c\* = wL\*):** {c_star:.2f} units
    * **Utility Level at Optimum (U\*):** {U_star:.3f}
    * **Optimality Condition Check:**
        * MRS = (θ * c\*) / l\* = ({theta:.2f} * {c_star:.2f}) / {l_star:.2f} = **{MRS_at_optimum:.3f}**
        * Wage (w) = **{w:.3f}**
        * (MRS should equal w at the optimum)
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
interact(
    labor_leisure_optimizer,
    w=FloatSlider(value=15.0, min=1, max=50, step=1, description='Wage Rate (w):', style=style, readout_format='.1f'),
    T=IntSlider(value=24, min=8, max=40, step=1, description='Time Endowment (T hrs):', style=style),
    theta=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Leisure Preference (theta θ):', style=style, readout_format='.1f'),
);


<>:126: SyntaxWarning: invalid escape sequence '\*'
<>:124: SyntaxWarning: invalid escape sequence '\*'
<>:124: SyntaxWarning: invalid escape sequence '\*'
<>:124: SyntaxWarning: invalid escape sequence '\*'
<>:125: SyntaxWarning: invalid escape sequence '\*'
<>:126: SyntaxWarning: invalid escape sequence '\*'
<>:124: SyntaxWarning: invalid escape sequence '\*'
<>:124: SyntaxWarning: invalid escape sequence '\*'
<>:124: SyntaxWarning: invalid escape sequence '\*'
<>:125: SyntaxWarning: invalid escape sequence '\*'
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_86910/3578401104.py:126: SyntaxWarning: invalid escape sequence '\*'
  
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_86910/3578401104.py:124: SyntaxWarning: invalid escape sequence '\*'
  display(Markdown(results_md))
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_86910/3578401104.py:124: SyntaxWarning: invalid escape sequence '\*'
  display(Markdown(results_md))
/var/folders/kz/cc2y6j3953z1b

interactive(children=(FloatSlider(value=15.0, description='Wage Rate (w):', max=50.0, min=1.0, readout_format=…